In [134]:
import ccxt
import pandas as pd

# Initialize Binance exchange
exchange = ccxt.binance()

# Fetch 1-minute OHLCV data
symbol = "BTC/USDT"
timeframe = "1m"  # Fetch 1-minute data
limit = 1000  # Fetch last 1000 minutes (~16.6 hours)

ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)

# Convert to DataFrame
df = pd.DataFrame(ohlcv, columns=["Timestamp", "Open", "High", "Low", "Close", "Volume"])
df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit="ms")
df.set_index("Timestamp", inplace=True)

# Resample to 10-minute data
df_10min = df.resample("10T").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume": "sum"
}).dropna()

# Resample to 4-hour data
df_4hour = df.resample("4H").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume": "sum"
}).dropna()

/var/folders/g9/01xw9sf55fbg7wx6rnn2dsc40000gn/T/ipykernel_4898/2536193416.py:20: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_10min = df.resample("10T").agg({
/var/folders/g9/01xw9sf55fbg7wx6rnn2dsc40000gn/T/ipykernel_4898/2536193416.py:29: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_4hour = df.resample("4H").agg({


In [135]:
# Fractal identification function
def identify_fractals(df):
    df["Bullish_Fractal"] = (df["Low"].shift(2) > df["Low"].shift(1)) & (df["Low"].shift(2) > df["Low"]) & \
                            (df["Low"].shift(2) > df["Low"].shift(-1)) & (df["Low"].shift(2) > df["Low"].shift(-2))

    df["Bearish_Fractal"] = (df["High"].shift(2) < df["High"].shift(1)) & (df["High"].shift(2) < df["High"]) & \
                            (df["High"].shift(2) < df["High"].shift(-1)) & (df["High"].shift(2) < df["High"].shift(-2))

    return df

# Apply fractal detection to 10-minute data
df_10min = identify_fractals(df_10min)

# Display fractals
fractals = df_10min[df_10min["Bullish_Fractal"] | df_10min["Bearish_Fractal"]]
print("\nDetected Fractals:\n", fractals[["High", "Low", "Bullish_Fractal", "Bearish_Fractal"]])



Detected Fractals:
                           High        Low  Bullish_Fractal  Bearish_Fractal
Timestamp                                                                  
2025-01-28 21:10:00  101547.22  101149.11            False             True
2025-01-28 21:40:00  101100.01  100598.22             True            False
2025-01-28 21:50:00  100734.85  100272.68             True            False
2025-01-28 22:00:00  100712.18  100274.96             True            False
2025-01-28 22:20:00  100965.72  100570.36            False             True
2025-01-28 22:50:00  101340.00  100923.07            False             True
2025-01-28 23:50:00  101450.00  101191.48            False             True
2025-01-29 00:00:00  101480.16  101328.01            False             True
2025-01-29 00:10:00  101587.49  101376.34            False             True
2025-01-29 00:20:00  101682.58  101476.73            False             True
2025-01-29 00:30:00  101654.00  101396.00            False         

In [136]:
# Reset index to make Timestamp a column
fractals = fractals.reset_index()
df_4hour = df_4hour.reset_index()

# Rename the index column for clarity, avoiding duplicate columns
fractals = fractals.rename(columns={"Timestamp": "Fractal_Timestamp"})
df_4hour = df_4hour.rename(columns={"Timestamp": "Fractal_Timestamp"})

# Ensure unique column names before merging
fractals.columns = [f"fractals_{col}" if col != "Fractal_Timestamp" else "Fractal_Timestamp" for col in fractals.columns]
df_4hour.columns = [f"df4hour_{col}" if col != "Fractal_Timestamp" else "Fractal_Timestamp" for col in df_4hour.columns]

# Perform asof merge
matched_patterns = pd.merge_asof(
    fractals.sort_values("Fractal_Timestamp"),
    df_4hour.sort_values("Fractal_Timestamp"),
    on="Fractal_Timestamp",
    direction="backward"
)

# Drop rows where first 3 candles don't match
matched_patterns = matched_patterns.dropna()

print("\nMatched Patterns in 4-Hour Data:\n", matched_patterns)



Matched Patterns in 4-Hour Data:
      Fractal_Timestamp  fractals_Open  fractals_High  fractals_Low  \
0  2025-01-28 21:10:00      101261.88      101547.22     101149.11   
1  2025-01-28 21:40:00      100888.01      101100.01     100598.22   
2  2025-01-28 21:50:00      100686.00      100734.85     100272.68   
3  2025-01-28 22:00:00      100334.00      100712.18     100274.96   
4  2025-01-28 22:20:00      100952.02      100965.72     100570.36   
5  2025-01-28 22:50:00      101250.01      101340.00     100923.07   
6  2025-01-28 23:50:00      101200.04      101450.00     101191.48   
7  2025-01-29 00:00:00      101335.52      101480.16     101328.01   
8  2025-01-29 00:10:00      101376.35      101587.49     101376.34   
9  2025-01-29 00:20:00      101539.93      101682.58     101476.73   
10 2025-01-29 00:30:00      101478.01      101654.00     101396.00   
11 2025-01-29 00:50:00      101718.72      101819.67     101645.75   
12 2025-01-29 01:00:00      101702.60      101993.68   

In [138]:
# Step 1: Prepare data for prediction
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Feature extraction from matched patterns
# Here, we assume that 'matched_patterns' has the fractal information

# Extracting previous candles and fractal features for training
X = matched_patterns[['fractals_Low', 'fractals_High', 'fractals_Bullish_Fractal', 'fractals_Bearish_Fractal',
                       'df4hour_Open', 'df4hour_High', 'df4hour_Low', 'df4hour_Close', 'df4hour_Volume']]

# The target variable - next 4-hour High and Low
y = matched_patterns[['df4hour_High', 'df4hour_Low']]

# Step 2: Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Build the model - Random Forest for regression
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 4: Evaluate the model
y_pred = model.predict(X_test)

# Display the predicted vs actual values
predicted_vs_actual = pd.DataFrame({
    'Predicted_High': y_pred[:, 0],
    'Actual_High': y_test.iloc[:, 0],
    'Predicted_Low': y_pred[:, 1],
    'Actual_Low': y_test.iloc[:, 1]
})

print(predicted_vs_actual.head())


    Predicted_High  Actual_High  Predicted_Low  Actual_Low
13     102325.0284    102319.69    101334.2850   101328.01
45     102836.7663    102842.04    101796.0375   101769.78
47     102777.3835    102842.04    101737.7622   101769.78
44     102838.3459    102842.04    101800.1462   101769.78
17     102330.3668    102319.69    101340.5600   101328.01


In [ ]:
# Step 5: Evaluate the model performance
from sklearn.metrics import mean_absolute_error, r2_score

# Calculate Mean Absolute Error (MAE) and R-squared (R2) score for evaluation
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nModel Evaluation:")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R2): {r2}")

# Step 6: Making predictions on new (live) data
# Suppose you get new live fractal data and 4-hour data for prediction
new_fractal_low = 35000  # New fractal low from latest data
new_fractal_high = 35500  # New fractal high from latest data
new_bullish_fractal = 1  # 1 if bullish fractal, else 0
new_bearish_fractal = 0  # 0 if bearish fractal, else 1

# Latest 4-hour data
new_open = 35200
new_high = 35600
new_low = 35050
new_close = 35400
new_volume = 1200

# Prepare the new data as a DataFrame
new_data = pd.DataFrame({
    'fractals_Low': [new_fractal_low],
    'fractals_High': [new_fractal_high],
    'fractals_Bullish_Fractal': [new_bullish_fractal],
    'fractals_Bearish_Fractal': [new_bearish_fractal],
    'df4hour_Open': [new_open],
    'df4hour_High': [new_high],
    'df4hour_Low': [new_low],
    'df4hour_Close': [new_close],
    'df4hour_Volume': [new_volume]
})

# Use the trained model to predict the high and low
predicted = model.predict(new_data)

# Output the predicted high and low
print(f"\nPredicted High: {predicted[0][0]}")
print(f"Predicted Low: {predicted[0][1]}")
 